In [7]:
import cv2
from os import path, listdir
from PIL import Image, ImageDraw
import numpy


In [8]:
data_path = "../train"

In [16]:
for f in listdir(data_path):
    image_folder_name = f.split("-")[0:2]
    image_folder = path.join('../../ssd.pytorch/data/MOTdevkit/',"-".join(image_folder_name), "img1")
    result_file = path.join("MDNet", f+".txt")
    video_name = path.join("video", f+"-MDNet"+".avi")
    len_seq = 0
    with open(path.join(data_path, f, "seqinfo.ini")) as seq:
        len_seq = int(seq.readlines()[4].strip().split("=")[1])
    getVideo(len_seq, image_folder, result_file, video_name)
    print(f)

MOT17-02-DPM


FileNotFoundError: [Errno 2] No such file or directory: 'MDNet\\MOT17-02-FRCNN.txt'

In [14]:
def getVideo(len_seq, image_folder, result_file, video_name):
    result = [[] for i in range(len_seq + 1)]

    results = open(result_file, "r")    
    for line in results:
        line = list(map(float, line.strip('\n').split(',')))
        result[int(line[0])].append(line)
    results.close()

    images = [img for img in listdir(image_folder) if img.endswith(".jpg")]
    frame = cv2.imread(path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc('M','J','P','G'), 30, (width,height))

    i = 1

    for image in images:
        img = Image.open(path.join(image_folder, image))
        dr = ImageDraw.Draw(img)
        for obj in result[i]:
            x, y, w, h = obj[2], obj[3], obj[4], obj[5]
            dr.rectangle(((x, y), (x+w , y+h)), outline="red")
            rectangle_width(dr,x,y,w,h,'red')
        i +=1
        pil_image = img.convert('RGB')
        open_cv_image = numpy.array(pil_image) 
        # Convert RGB to BGR 
        open_cv_image = open_cv_image[:, :, ::-1].copy() 
        video.write(open_cv_image)
        img.close()

    cv2.destroyAllWindows()
    video.release()
    

In [10]:
def rectangle_width(image, x, y, w, h, outline, width=5):
    for step in range(width):
        image.rectangle(((x+step, y+step), (x+w+step, y+h+step)), outline=outline)